In [1]:
from VisitorMC3 import *
import ast
import os
import pandas as pd
import csv
from collections import defaultdict

# Caminhos
csv_questoes = "../Etapa_3/questoes_ordenadas.csv"
pasta_usuarios = "../Etapa_3/UsuariosUnicos"
output_summary = "misconceptions_summary_por_questao.csv"
output_detailed = "misconceptions_detailed_por_usuario.csv"

# Constants for MC³ detection
C4_MAX_ALLOWED_RANGEITER = 50
E2_MAX_ALLOWED_LISTS = 5
G4_MIN_VAR_CHRS = 4
G4_MIN_FNC_CHRS = 8
G4_MAX_ALLOWED_NONSIGNIFICANT = 70

# Lista de todos os MC³ possíveis
MC3_TYPES = ['A4', 'A5', 'B4', 'B6', 'B8', 'B9', 'B12', 'C1', 'C2', 'C3', 'C4', 'C8', 'D4', 'E2', 'G4', 'G5', 'H1']

def analisar_codigo(filepath):
    """Analisa um arquivo Python e retorna os MC³ detectados"""
    try:
        with open(filepath, 'r', encoding="utf-8") as file:
            code = file.read()
        parsed = ast.parse(code)
    except Exception as e:
        print(f"Erro ao fazer parsing de {filepath}: {e}")
        return []

    visitor = VisitorMC3()
    
    try:
        # Executar todas as detecções
        resultados = {}
        
        resA4 = visitor.getA4(parsed)
        resA5 = visitor.getA5(parsed)
        resB4 = visitor.getB4(parsed)
        resB6 = visitor.getB6(parsed)
        resB8 = visitor.getB8(parsed)
        resB9 = visitor.getB9(parsed)
        resB12 = visitor.getB12(parsed)
        resC1 = visitor.getC1(parsed)
        resC2 = visitor.getC2(parsed)
        resC3 = visitor.getC3(parsed)
        resC4 = visitor.getC4(parsed, C4_MAX_ALLOWED_RANGEITER)
        resC8 = visitor.getC8(parsed)
        resD4 = visitor.getD4(parsed)
        resE2 = visitor.getE2(parsed, E2_MAX_ALLOWED_LISTS)
        resG4 = visitor.getG4(parsed, G4_MIN_VAR_CHRS, G4_MIN_FNC_CHRS, G4_MAX_ALLOWED_NONSIGNIFICANT)
        resG5 = visitor.getG5(parsed)
        resH1 = visitor.getH1(parsed)

        # Organizar resultados - alguns retornam tuplas, outros boolean
        res_map = {
            'A4': resA4[0] if isinstance(resA4, tuple) else resA4,
            'A5': resA5[0] if isinstance(resA5, tuple) else resA5,
            'B4': resB4,
            'B6': resB6,
            'B8': resB8,
            'B9': resB9,
            'B12': resB12,
            'C1': resC1,
            'C2': resC2,
            'C3': resC3,
            'C4': resC4,
            'C8': resC8,
            'D4': resD4,
            'E2': resE2,
            'G4': resG4,
            'G5': resG5,
            'H1': resH1
        }
        
        # Retornar apenas os MC³ detectados (True)
        detectados = [mc3 for mc3, resultado in res_map.items() if resultado]
        return detectados
        
    except Exception as e:
        print(f"Erro ao processar {filepath}: {e}")
        return []

def main():
    # Ler o CSV de questões
    print("Lendo questoes_ordenadas.csv...")
    df_questoes = pd.read_csv(csv_questoes)
    
    # Estruturas para armazenar os resultados
    summary_data = []  # Para o CSV resumo por questão
    detailed_data = []  # Para o CSV detalhado por usuário
    
    # Processar cada questão
    for _, row in df_questoes.iterrows():
        questao_id = row['id']
        usuarios_str = row['usuarios_respondidos']
        
        print(f"\nProcessando questão {questao_id}...")
        
        # Converter string de usuários para lista
        usuarios = [u.strip() for u in usuarios_str.split(',')]
        
        # Contadores para esta questão
        mc3_counters = defaultdict(int)
        usuarios_analisados = 0
        usuarios_com_misconceptions = 0
        
        # Analisar cada usuário desta questão
        for usuario_id in usuarios:
            # Procurar arquivo do usuário (assumindo que está na pasta do usuário)
            usuario_pasta = os.path.join(pasta_usuarios, str(usuario_id))
            
            if not os.path.exists(usuario_pasta):
                continue
                
            # Procurar arquivo Python na pasta do usuário
            py_files = []
            for item in os.listdir(usuario_pasta):
                item_path = os.path.join(usuario_pasta, item)
                if item.endswith('.py'):
                    py_files.append(item_path)
                elif os.path.isdir(item_path):
                    # Procurar em subpastas também
                    for subitem in os.listdir(item_path):
                        if subitem.endswith('.py'):
                            py_files.append(os.path.join(item_path, subitem))
            
            # Analisar todos os arquivos Python encontrados para este usuário
            usuario_misconceptions = set()
            
            for py_file in py_files:
                misconceptions = analisar_codigo(py_file)
                usuario_misconceptions.update(misconceptions)
            
            if usuario_misconceptions:
                usuarios_analisados += 1
                usuarios_com_misconceptions += 1
                
                # Atualizar contadores
                for mc3 in usuario_misconceptions:
                    mc3_counters[mc3] += 1
                
                # Adicionar ao CSV detalhado
                detailed_data.append({
                    'questao_id': questao_id,
                    'usuario_id': usuario_id,
                    'misconceptions_detectados': ','.join(sorted(usuario_misconceptions)),
                    'total_misconceptions': len(usuario_misconceptions)
                })
            elif py_files:  # Usuario tinha arquivos mas sem misconceptions
                usuarios_analisados += 1
                detailed_data.append({
                    'questao_id': questao_id,
                    'usuario_id': usuario_id,
                    'misconceptions_detectados': '',
                    'total_misconceptions': 0
                })
        
        # Criar linha do resumo para esta questão
        summary_row = {
            'questao_id': questao_id,
            'total_usuarios_responderam': len(usuarios),
            'usuarios_analisados': usuarios_analisados,
            'usuarios_com_misconceptions': usuarios_com_misconceptions,
            'percentual_com_misconceptions': round((usuarios_com_misconceptions/usuarios_analisados*100) if usuarios_analisados > 0 else 0, 2)
        }
        
        # Adicionar contadores de cada MC³
        for mc3_type in MC3_TYPES:
            summary_row[f'count_{mc3_type}'] = mc3_counters[mc3_type]
            summary_row[f'perc_{mc3_type}'] = round((mc3_counters[mc3_type]/usuarios_analisados*100) if usuarios_analisados > 0 else 0, 2)
        
        summary_data.append(summary_row)
        
        print(f"  - Usuários analisados: {usuarios_analisados}/{len(usuarios)}")
        print(f"  - Com misconceptions: {usuarios_com_misconceptions}")
        if mc3_counters:
            print(f"  - MC³ mais frequentes: {dict(sorted(mc3_counters.items(), key=lambda x: x[1], reverse=True)[:3])}")
    
    # Salvar CSV resumo por questão
    print(f"\nSalvando resumo em {output_summary}...")
    df_summary = pd.DataFrame(summary_data)
    df_summary.to_csv(output_summary, index=False, encoding='utf-8')
    
    # Salvar CSV detalhado por usuário
    print(f"Salvando detalhes em {output_detailed}...")
    df_detailed = pd.DataFrame(detailed_data)
    df_detailed.to_csv(output_detailed, index=False, encoding='utf-8')
    
    print("\nResumo geral:")
    print(f"- Total de questões processadas: {len(summary_data)}")
    print(f"- Total de análises de usuários: {len(detailed_data)}")
    print(f"- Arquivos gerados: {output_summary}, {output_detailed}")
    
    # Mostrar estatísticas gerais
    if summary_data:
        total_usuarios_analisados = sum(row['usuarios_analisados'] for row in summary_data)
        total_com_misconceptions = sum(row['usuarios_com_misconceptions'] for row in summary_data)
        
        print(f"- Total usuários analisados: {total_usuarios_analisados}")
        print(f"- Total com misconceptions: {total_com_misconceptions}")
        print(f"- Percentual geral com misconceptions: {round(total_com_misconceptions/total_usuarios_analisados*100, 2)}%")
        
        # MC³ mais frequentes globalmente
        global_mc3_counts = defaultdict(int)
        for row in summary_data:
            for mc3_type in MC3_TYPES:
                global_mc3_counts[mc3_type] += row[f'count_{mc3_type}']
        
        top_mc3 = sorted(global_mc3_counts.items(), key=lambda x: x[1], reverse=True)[:5]
        print(f"- Top 5 MC³ mais frequentes: {dict(top_mc3)}")

if __name__ == "__main__":
    main()

Lendo questoes_ordenadas.csv...

Processando questão 1642...
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9739/codes/5817_6196.py: invalid syntax (<unknown>, line 10)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9739/codes/5730_1949.py: invalid syntax (<unknown>, line 6)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9739/codes/5874_3591.py: expected an indented block after 'if' statement on line 7 (<unknown>, line 7)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9739/codes/5676_652.py: invalid syntax (<unknown>, line 1)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9739/codes/5676_1092.py: invalid syntax (<unknown>, line 6)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9739/codes/5764_6783.py: expected an indented block after 'if' statement on line 4 (<unknown>, line 5)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9684/codes/5608_825.py: invalid syntax (<unknown>, line 1)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9684/codes/5708_5946.py: c

<unknown>:4: SyntaxWarning: invalid decimal literal


Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9693/codes/5608_2895.py: invalid syntax (<unknown>, line 3)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9662/codes/5678_652.py: invalid syntax (<unknown>, line 2)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9662/codes/5798_6580.py: unexpected indent (<unknown>, line 4)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9827/codes/5896_1227.py: invalid syntax (<unknown>, line 8)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9827/codes/5726_1102.py: invalid syntax (<unknown>, line 8)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9827/codes/5842_2101.py: invalid decimal literal (<unknown>, line 1)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9827/codes/5896_1223.py: invalid syntax (<unknown>, line 2)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9752/codes/5935_1645.py: '(' was never closed (<unknown>, line 3)
  - Usuários analisados: 19/19
  - Com misconceptions: 19
  - MC³ mais frequentes: {'A5': 19, '

<unknown>:4: SyntaxWarning: invalid decimal literal


Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9693/codes/5608_2895.py: invalid syntax (<unknown>, line 3)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/7586/codes/5764_6749.py: invalid syntax (<unknown>, line 2)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9707/codes/5744_6915.py: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<unknown>, line 4)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9707/codes/5865_3589.py: invalid syntax (<unknown>, line 1)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9707/codes/5905_6336.py: invalid syntax (<unknown>, line 6)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9707/codes/5904_1644.py: invalid syntax (<unknown>, line 1)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9707/codes/5744_6094.py: invalid syntax (<unknown>, line 4)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9707/codes/5766_6933.py: invalid syntax (<unknown>, line 9)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9707/codes/5904

...

Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9709/codes/5744_6914.py: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<unknown>, line 4)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9709/codes/5622_4236.py: invalid syntax (<unknown>, line 6)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9709/codes/5693_2971.py: '(' was never closed (<unknown>, line 6)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5746_6104.py: invalid character '√' (U+221A) (<unknown>, line 8)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5687_2445.py: expected an indented block after 'if' statement on line 4 (<unknown>, line 5)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5687_2969.py: expected ':' (<unknown>, line 5)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5889_2562.py: invalid syntax (<unknown>, line 9)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5889_6354.py: invalid syntax (<unknown>, line 6)
Err

<unknown>:4: SyntaxWarning: invalid decimal literal


Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9693/codes/5608_2895.py: invalid syntax (<unknown>, line 3)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9834/codes/5668_2445.py: invalid syntax (<unknown>, line 6)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9834/codes/5668_1397.py: '(' was never closed (<unknown>, line 1)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5746_6104.py: invalid character '√' (U+221A) (<unknown>, line 8)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5687_2445.py: expected an indented block after 'if' statement on line 4 (<unknown>, line 5)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5687_2969.py: expected ':' (<unknown>, line 5)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5889_2562.py: invalid syntax (<unknown>, line 9)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9680/codes/5889_6354.py: invalid syntax (<unknown>, line 6)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnico

<unknown>:4: SyntaxWarning: invalid decimal literal


Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9876/codes/5896_2562.py: invalid octal literal (<unknown>, line 33)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9876/codes/5670_1092.py: invalid syntax (<unknown>, line 5)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9876/codes/5841_7028.py: invalid syntax (<unknown>, line 5)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9876/codes/5781_2518.py: invalid syntax (<unknown>, line 1)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9876/codes/5727_6100.py: '(' was never closed (<unknown>, line 7)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9876/codes/5782_6048.py: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<unknown>, line 5)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9778/codes/5681_1092.py: invalid syntax (<unknown>, line 1)
Erro ao fazer parsing de ../Etapa_3/UsuariosUnicos/9778/codes/5681_1055.py: expected an indented block after 'if' statement on line 12 (<unknown>, line 15)
Erro ao